# Semantic Analysis Pipeline

This notebook demonstrates the semantic text analysis capabilities using our custom analyzers.

## Setup
Import required packages and configure the environment:



In [1]:
# At start of notebook
import sys
from pathlib import Path
import logging
import os

# Add project root to Python path
project_root = str(Path().resolve().parent)
if project_root not in sys.path:
    sys.path.append(project_root)

In [2]:
# import logging
# from src.nb_helpers.logging import configure_logging

# Set up environment with DEBUG level
from src.nb_helpers.environment import setup_notebook_env, verify_environment
setup_notebook_env(log_level="DEBUG")

# Any verification needed will maintain DEBUG level
verify_environment(log_level="DEBUG")

2024-11-23 10:04:59,236 - src.utils.FileUtils.file_utils - DEBUG [file_utils.py:192] - Initialized FileUtils with log level: INFO
2024-11-23 10:04:59,236 - src.utils.FileUtils.file_utils - DEBUG [file_utils.py:198] - Project root: c:\Users\tja\OneDrive - Rastor-instituutti ry\Tiedostot\Rastor-instituutti\kehittäminen\analytiikka\repos\semantic-text-analyzer
2024-11-23 10:04:59,244 - src.utils.FileUtils.file_utils - DEBUG [file_utils.py:192] - Initialized FileUtils with log level: INFO
2024-11-23 10:04:59,244 - src.utils.FileUtils.file_utils - DEBUG [file_utils.py:198] - Project root: c:\Users\tja\OneDrive - Rastor-instituutti ry\Tiedostot\Rastor-instituutti\kehittäminen\analytiikka\repos\semantic-text-analyzer


Environment Check Results:

Basic Setup:
-----------
✓ Project root in path
✓ FileUtils initialized
✓ .env file loaded

Environment Variables:
---------------------
✓ OPENAI_API_KEY set
✓ ANTHROPIC_API_KEY set

Project Structure:
-----------------
✓ Raw data exists
✓ Processed data exists
✓ Configuration exists
✓ Main config.yaml exists

Environment Status: Ready ✓


True

In [3]:
# Import necessary components
from src.loaders.parameter_handler import ParameterHandler
from src.nb_helpers.analyzers import (
    analyze_keywords,
    analyze_themes,
    analyze_categories,
    analyze_text,
    AnalysisOptions
)

from scripts.migrate_parameters import create_example_parameters
from src.nb_helpers.logging import configure_logging, verify_logging_setup_with_hierarchy, reset_debug_logging
from src.loaders.parameter_handler import ParameterHandler, get_parameter_file_path, verify_parameter_file



In [4]:
# Set initial logging
configure_logging(level="DEBUG")
# Keep HTTP loggers at INFO
for name in ["httpx", "httpcore", "openai", "anthropic"]:
    logging.getLogger(name).setLevel(logging.INFO)
    
# verify_logging_setup_with_hierarchy()

2024-11-23 10:04:59,392 - src.nb_helpers.logging - DEBUG - Logging configured at DEBUG level


In [5]:
# detailed_logging_info = True
# if detailed_logging_info:
#     from src.nb_helpers.logging import verify_logging_setup_with_hierarchy
#     # Configure logging
#     # configure_logging(level="DEBUG")
#     # Verify with detailed information
#     verify_logging_setup_with_hierarchy()


In [6]:
# Example texts in different languages
example_texts = {
    "English Technical": """
        The cloud migration project improved system scalability while reducing costs.
        New DevOps practices streamlined the deployment pipeline significantly.
    """,
    
    "Finnish Technical": """
        Pilvipalveluihin siirtyminen paransi järjestelmän skaalautuvuutta ja vähensi kustannuksia.
        Uudet DevOps-käytännöt tehostivat merkittävästi käyttöönottoprosessia.
    """,
    
    "English Business": """
        Q3 financial results show 15% revenue growth and improved profit margins.
        Customer acquisition costs decreased while retention rates increased.
    """,
    
    "Finnish Business": """
        Q3 taloudelliset tulokset osoittavat 15% liikevaihdon kasvun ja parantuneet katteet.
        Asiakashankinnan kustannukset laskivat ja asiakaspysyvyys parani.
    """
}



In [7]:
# Create and load parameters
params_file_name = "parameters_en.xlsx"

# Get the full parameter file path
params_file = get_parameter_file_path(params_file_name)

# Create file if it doesn't exist
if not params_file.exists():
    params_file = create_example_parameters(params_file_name)
    print(f"Created parameter file at: {params_file}")
else:
    print(f"Using existing parameter file at: {params_file}")

# Verify the file
verify_parameter_file(params_file)

# Load parameters
handler = ParameterHandler(params_file_name)  # Can now use just the file name
params = handler.get_parameters()



2024-11-23 10:04:59,484 - src.utils.FileUtils.file_utils - DEBUG [file_utils.py:192] - Initialized FileUtils with log level: INFO
2024-11-23 10:04:59,484 - src.utils.FileUtils.file_utils - DEBUG - Initialized FileUtils with log level: INFO
2024-11-23 10:04:59,488 - src.utils.FileUtils.file_utils - DEBUG [file_utils.py:198] - Project root: c:\Users\tja\OneDrive - Rastor-instituutti ry\Tiedostot\Rastor-instituutti\kehittäminen\analytiikka\repos\semantic-text-analyzer
2024-11-23 10:04:59,488 - src.utils.FileUtils.file_utils - DEBUG - Project root: c:\Users\tja\OneDrive - Rastor-instituutti ry\Tiedostot\Rastor-instituutti\kehittäminen\analytiikka\repos\semantic-text-analyzer


Using existing parameter file at: c:\Users\tja\OneDrive - Rastor-instituutti ry\Tiedostot\Rastor-instituutti\kehittäminen\analytiikka\repos\semantic-text-analyzer\data\parameters\parameters_en.xlsx

Parameter File Verification:
Absolute path: C:\Users\tja\OneDrive - Rastor-instituutti ry\Tiedostot\Rastor-instituutti\kehittäminen\analytiikka\repos\semantic-text-analyzer\data\parameters\parameters_en.xlsx
File exists: True


2024-11-23 10:05:01,176 - src.utils.FileUtils.file_utils - DEBUG [file_utils.py:192] - Initialized FileUtils with log level: INFO



Found sheets:

General Parameters:
            parameter                                    value  \
0        max_keywords                                        8   
1            language                                       en   
2            focus_on  business and technical content analysis   
3  min_keyword_length                                        3   
4   include_compounds                                     True   

                   description  
0  Maximum keywords to extract  
1             Content language  
2          Analysis focus area  
3       Minimum keyword length  
4        Handle compound words  

Predefined Keywords:
                 keyword  importance     domain
0       machine learning         1.0  technical
1          data analysis         0.9  technical
2        cloud computing         0.8  technical
3  business intelligence         0.9   business
4        market analysis         0.8   business

Excluded Keywords:
  keyword       reason
0     the  Commo

2024-11-23 10:05:01,176 - src.utils.FileUtils.file_utils - DEBUG - Initialized FileUtils with log level: INFO
2024-11-23 10:05:01,176 - src.utils.FileUtils.file_utils - DEBUG [file_utils.py:198] - Project root: c:\Users\tja\OneDrive - Rastor-instituutti ry\Tiedostot\Rastor-instituutti\kehittäminen\analytiikka\repos\semantic-text-analyzer
2024-11-23 10:05:01,176 - src.utils.FileUtils.file_utils - DEBUG - Project root: c:\Users\tja\OneDrive - Rastor-instituutti ry\Tiedostot\Rastor-instituutti\kehittäminen\analytiikka\repos\semantic-text-analyzer
2024-11-23 10:05:01,176 - src.utils.FileUtils.file_utils - DEBUG [file_utils.py:198] - Project root: c:\Users\tja\OneDrive - Rastor-instituutti ry\Tiedostot\Rastor-instituutti\kehittäminen\analytiikka\repos\semantic-text-analyzer
2024-11-23 10:05:01,176 - src.utils.FileUtils.file_utils - DEBUG - Project root: c:\Users\tja\OneDrive - Rastor-instituutti ry\Tiedostot\Rastor-instituutti\kehittäminen\analytiikka\repos\semantic-text-analyzer
2024-11-23

In [8]:
print("\nLoaded parameters:")
# params.print()  # Uses the new print method

# Or just
print(params)  # Uses the new __str__ method


Loaded parameters:
ParameterSet(
  general=max_keywords=8 min_keyword_length=3 language='en' focus_on='business and technical content analysis' include_compounds=True max_themes=3 min_confidence=0.3 column_name_to_analyze='content',
  categories=6 items,
  predefined_keywords=6 items,
  excluded_keywords=6 items,
  analysis_settings=theme_analysis=ThemeAnalysisSettings(enabled=True, min_confidence=0.5) weights=AnalysisWeights(statistical=0.4, llm=0.6),
  domain_context=3 items
)


In [9]:
# Create example files

create_new_example_files=False

if create_new_example_files:
    en_params = create_example_parameters("parameters_en.xlsx", "en")
    fi_params = create_example_parameters("parameters_fi.xlsx", "fi")
    print(f"Created parameter files:\n- {en_params}\n- {fi_params}")
    print(f"Created parameter files:\n- {en_params}\n- {fi_params}")
else:
    print("Not creating new example files.")

Not creating new example files.


In [11]:
# # Check parameter validation
# print("Parameter Validation:")
# print("-" * 50)
# is_valid, warnings, errors = handler.validate()
# if warnings:
#     print("\nWarnings:")
#     for warning in warnings:
#         print(f"- {warning}")
# if not is_valid:
#     print("\nErrors:")
#     for error in errors:
#         print(f"- {error}")
# else:
#     print("\nParameters validated successfully!")

In [12]:
# Test cases
test_texts = {
    "Technical Compound Terms": """
        The cloud migration project improved system scalability.
        DevOps practices streamlined the deployment pipeline.
        Our microservices architecture enables API integrations.
    """,
    
    "Mixed Domain Content": """
        The IT department's infrastructure costs decreased by 25%
        after implementing cloud-native solutions. Monthly recurring
        revenue from SaaS products grew steadily while deployment
        frequency improved.
    """,
    
    "Business Focus": """
        Market analysis shows 15% revenue growth in Q3.
        Customer acquisition costs decreased while retention rates
        increased. Strategic partnerships drove innovation.
    """,
    
    "Multiple Compounds": """
        Machine learning models process real-time data streams.
        The CI/CD pipeline integrates automated testing workflows.
        Cloud-based infrastructure supports multi-region deployments.
    """
}

finnish_texts = {
    "technical_fi_1":"Pilvipalveluiden käyttöönotto tehosti järjestelmän skaalautuvuutta merkittävästi. DevOps-prosessit nopeuttivat julkaisusykliä ja automatisoivat laadunvarmistusta. Kuukausittainen tilaustuotto SaaS-ratkaisuista kasvoi 25%.",
    "technical_fi_2":"Mikropalveluarkkitehtuuri mahdollisti järjestelmän modulaarisen kehityksen. Konttiteknologian avulla saavutettiin parempi resurssien käyttöaste ja joustavampi ylläpito. Rajapintojen dokumentointi helpotti integraatioiden toteuttamista.",
    "technical_fi_3":"Tekoälypohjaiset ennusteet auttoivat optimoimaan kuormantasausta. Pilvinatiivi lähestymistapa vähensi infrastruktuurikustannuksia ja paransi vikasietoisuutta. Monitorointi tarjosi reaaliaikaista näkyvyyttä suorituskykyyn.",
    "business_fi_1":"Liikevaihdon kasvu vahvistui kolmannella vuosineljänneksellä 15 prosenttiin. Asiakashankinnan kustannukset laskivat samalla kun asiakaspysyvyys parani. Markkinaosuus kasvoi erityisesti pilvipalveluiden segmentissä.",
    "business_fi_2":"Analytiikkatyökalut paljastivat uusia käyttäytymismalleja asiakasrajapinnassa. Toistuvaislaskutuksen osuus kokonaistuotoista nousi 75 prosenttiin. Automaattinen raportointi tehosti päätöksentekoa.",
    "business_fi_3":"Uudet tuotelanseeraukset vahvistivat kilpailuasemaa. Strategiset kumppanuudet mahdollistivat laajentumisen uusille markkina-alueille. Resurssien kohdentaminen tuotekehitykseen tuotti merkittävää kasvua."
}



In [13]:
from src.analyzers import KeywordAnalyzer, ThemeAnalyzer, CategoryAnalyzer #, TextAnalyzer
from src.core.language_processing import create_text_processor

# from src.analyzers import CategoryAnalyzer
# from src.core.language_processing import create_text_processor
from src.loaders.models import CategoryConfig

### Category Analysis

In [32]:
# Create language processor
processor_fi = create_text_processor(language="fi")
# Create language processor
processor_en = create_text_processor(language="en")


2024-11-23 10:16:06,983 - src.core.language_processing.factory - DEBUG - Using default configuration
2024-11-23 10:16:06,987 - src.core.language_processing.factory - DEBUG - Creating text processor for language: fi
2024-11-23 10:16:06,988 - src.core.language_processing.factory - DEBUG - Creating text processor for language: fi
2024-11-23 10:16:06,989 - src.core.language_processing.factory - DEBUG - Creating fi processor
2024-11-23 10:16:06,994 - src.utils.FileUtils.file_utils - DEBUG [file_utils.py:192] - Initialized FileUtils with log level: INFO
2024-11-23 10:16:06,994 - src.utils.FileUtils.file_utils - DEBUG - Initialized FileUtils with log level: INFO
2024-11-23 10:16:06,996 - src.utils.FileUtils.file_utils - DEBUG [file_utils.py:198] - Project root: c:\Users\tja\OneDrive - Rastor-instituutti ry\Tiedostot\Rastor-instituutti\kehittäminen\analytiikka\repos\semantic-text-analyzer
2024-11-23 10:16:06,987 - src.core.language_processing.factory - DEBUG - Creating text processor for langu

In [33]:

# Define categories
categories = {
    "technical_infrastructure": CategoryConfig(
        description="Technical infrastructure and cloud solutions",
        keywords=["pilvipalvelu", "infrastruktuuri", "skaalautuvuus"],
        threshold=0.5
    ),
    "business_performance": CategoryConfig(
        description="Business performance metrics",
        keywords=["liikevaihto", "kasvu", "markkinaosuus"],
        threshold=0.5
    )
}



In [34]:
# test_case = "technical_fi_1"
test_case = "business_fi_1"
text= finnish_texts[test_case]



# test_case = "technical_fi_1"
# test_case = "Mixed Domain Content"
# text= test_texts[test_case]

In [35]:
# Initialize analyzer
cat_analyzer_fi = CategoryAnalyzer(
    categories=categories,
    language_processor=processor_fi,
    config={"min_confidence": 0.3}
)

# Analyze text
results = await cat_analyzer_fi.analyze(text)

# Display results
cat_analyzer_fi.display_categories(results)

2024-11-23 10:17:57,494 - src.core.language_processing.finnish - DEBUG - Base form for Liikevaihdon: liikevaihto
2024-11-23 10:17:57,496 - src.core.language_processing.finnish - DEBUG - Base form for kasvu: kasvu
2024-11-23 10:17:57,497 - src.core.language_processing.finnish - DEBUG - Base form for vuosineljänneksellä: vuosineljännes
2024-11-23 10:17:57,499 - src.core.language_processing.finnish - DEBUG - Base form for Asiakashankinnan: asiakashankinta
2024-11-23 10:17:57,500 - src.core.language_processing.finnish - DEBUG - Base form for kustannukset: kustannus
2024-11-23 10:17:57,502 - src.core.language_processing.finnish - DEBUG - Base form for asiakaspysyvyys: asiakaspysyvyys
2024-11-23 10:17:57,503 - src.core.language_processing.finnish - DEBUG - Base form for parani: para
2024-11-23 10:17:57,505 - src.core.language_processing.finnish - DEBUG - Base form for Markkinaosuus: markkinaosuus
2024-11-23 10:17:57,507 - src.core.language_processing.finnish - DEBUG - Base form for pilvipalv


Category Analysis Results

Category: business_performance
--------------------
Confidence: 0.95

Evidence:
- Liikevaihdon kasvu vahvistui kolmannella vuosineljänneksellä 15 prosenttiin. (relevance: 0.90)
- Markkinaosuus kasvoi erityisesti pilvipalveluiden segmentissä. (relevance: 0.90)

Related Themes: revenue growth, market share

Category: technical_infrastructure
--------------------
Confidence: 0.70

Evidence:
- Markkinaosuus kasvoi erityisesti pilvipalveluiden segmentissä. (relevance: 0.80)

Related Themes: cloud services, infrastructure


### Theme analysis

In [17]:
technical_text = """
Pilvipalveluiden käyttöönotto tehosti järjestelmän skaalautuvuutta merkittävästi. 
DevOps-prosessit nopeuttivat julkaisusykliä ja automatisoivat laadunvarmistusta. 
Kuukausittainen tilaustuotto SaaS-ratkaisuista kasvoi 25%.
"""

In [18]:
technical_text_2 = "Koneoppimismalleja koulutetaan suurilla datajoukolla tunnistamaan kaavoja. Neuroverkon arkkitehtuuri sisältää useita kerroksia piirteiden erottamiseen. Datan esikäsittely ja piirteiden suunnittelu ovat keskeisiä vaiheita prosessissa."

In [36]:
# Create language processor
# processor_en = create_text_processor(language="en")

# Initialize analyzer
theme_analyzer_en = ThemeAnalyzer(
    language_processor=processor_en,
    config={
        "max_themes": 3,
        "min_confidence": 0.3,
        "focus_on": "technical"
    }
)

# Create language processor
# processor_fi = create_text_processor(language="fi")

# Initialize analyzer
theme_analyzer_fi = ThemeAnalyzer(
    language_processor=processor_fi,
    config={
        "max_themes": 3,
        "min_confidence": 0.3,
        "focus_on": "technical"
    }
)



In [37]:
# test_case = "technical_fi_1"
test_case = "business_fi_1"
text= finnish_texts[test_case]



# test_case = "technical_fi_1"
# test_case = "Mixed Domain Content"
# text= test_texts[test_case]

In [38]:
# Analyze text
# results = await analyzer.analyze(technical_text)
results = await theme_analyzer_fi.analyze(text)

# Display results
theme_analyzer_fi.display_themes(results)

2024-11-23 10:21:56,121 - src.core.language_processing.finnish - DEBUG - Base form for Liikevaihdon: liikevaihto
2024-11-23 10:21:56,124 - src.core.language_processing.finnish - DEBUG - Base form for kasvu: kasvu
2024-11-23 10:21:56,125 - src.core.language_processing.finnish - DEBUG - Base form for vuosineljänneksellä: vuosineljännes
2024-11-23 10:21:56,127 - src.core.language_processing.finnish - DEBUG - Base form for Asiakashankinnan: asiakashankinta
2024-11-23 10:21:56,128 - src.core.language_processing.finnish - DEBUG - Base form for kustannukset: kustannus


2024-11-23 10:21:56,131 - src.core.language_processing.finnish - DEBUG - Base form for asiakaspysyvyys: asiakaspysyvyys
2024-11-23 10:21:56,133 - src.core.language_processing.finnish - DEBUG - Base form for parani: para
2024-11-23 10:21:56,135 - src.core.language_processing.finnish - DEBUG - Base form for Markkinaosuus: markkinaosuus
2024-11-23 10:21:56,137 - src.core.language_processing.finnish - DEBUG - Base form for pilvipalveluiden: pilvipalvelu
2024-11-23 10:21:56,138 - src.core.language_processing.finnish - DEBUG - Base form for segmentissä: segmentti
2024-11-23 10:22:05,204 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



Theme Analysis Results

Theme: Liikevaihdon kasvu
--------------------
Description: Liikevaihdon kasvu viittaa yrityksen myynnin ja taloudellisen suorituskyvyn parantumiseen, mikä on keskeinen indikaattori liiketoiminnan menestykselle.
Confidence: 0.95
Keywords: liikevaihto, kasvu, vuosineljännes

Evidence:
- Liikevaihdon kasvu vahvistui kolmannella vuosineljänneksellä 15 prosenttiin. (relevance: 0.90)

Theme: Asiakashankinnan tehokkuus
--------------------
Description: Asiakashankinnan tehokkuus käsittää asiakashankinnan kustannusten laskun ja asiakaspysyvyyden parantumisen, mikä osoittaa yrityksen kyvyn houkutella ja pitää asiakkaita.
Confidence: 0.90
Keywords: asiakashankinta, kustannus, asiakaspysyvyys

Evidence:
- Asiakashankinnan kustannukset laskivat samalla kun asiakaspysyvyys parani. (relevance: 0.90)

Theme: Markkinaosuus pilvipalveluissa
--------------------
Description: Markkinaosuuden kasvu pilvipalveluiden segmentissä viittaa yrityksen kilpailukyvyn parantumiseen ja sen 

In [39]:
# Testing function
async def test_keyword_analyzer(text: str, show_debug: bool = True, language: str = "en"):
    options = AnalysisOptions(
        show_confidence=True,
        show_evidence=True,
        show_keywords=True,
        show_raw_data=show_debug,
        debug_mode=True,
        language=language
    )
    
    results = await analyze_keywords(text, options)
    return results



In [40]:
results = await test_keyword_analyzer(technical_text_2, language="fi")

2024-11-23 10:22:37,073 - src.nb_helpers.analyzers - DEBUG - Starting keyword analysis
2024-11-23 10:22:37,076 - src.utils.FileUtils.file_utils - DEBUG [file_utils.py:192] - Initialized FileUtils with log level: INFO
2024-11-23 10:22:37,076 - src.utils.FileUtils.file_utils - DEBUG - Initialized FileUtils with log level: INFO
2024-11-23 10:22:37,079 - src.utils.FileUtils.file_utils - DEBUG [file_utils.py:198] - Project root: c:\Users\tja\OneDrive - Rastor-instituutti ry\Tiedostot\Rastor-instituutti\kehittäminen\analytiikka\repos\semantic-text-analyzer
2024-11-23 10:22:37,079 - src.utils.FileUtils.file_utils - DEBUG - Project root: c:\Users\tja\OneDrive - Rastor-instituutti ry\Tiedostot\Rastor-instituutti\kehittäminen\analytiikka\repos\semantic-text-analyzer
2024-11-23 10:22:37,085 - src.loaders.parameter_handler - DEBUG - No parameter file specified, using defaults
2024-11-23 10:22:37,086 - src.nb_helpers.analyzers - DEBUG - Initialized TextAnalyzer with options: AnalysisOptions(show_co


Keywords Found:
  • koneoppimismalli     [███████████████████░] (0.95)
  • datajoukko           [███████████████████░] (0.95)
  • neuroverkon arkkitehtuuri [███████████████████░] (0.95)
  • datan esikäsittely   [███████████████████░] (0.95)
  • piirteiden suunnittelu [███████████████████░] (0.95)

Debug Information:
--------------------
{
  "language": "fi",
  "error": null,
  "success": true,
  "keywords": [
    {
      "keyword": "koneoppimismalli",
      "score": 0.95,
      "domain": "technical",
      "compound_parts": [
        "kone",
        "oppia",
        "malli"
      ]
    },
    {
      "keyword": "datajoukko",
      "score": 0.95,
      "domain": "technical",
      "compound_parts": [
        "data",
        "joukko"
      ]
    },
    {
      "keyword": "neuroverkon arkkitehtuuri",
      "score": 0.95,
      "domain": "technical",
      "compound_parts": null
    },
    {
      "keyword": "datan esik\u00e4sittely",
      "score": 0.95,
      "domain": "technical",
    

In [41]:
# test_case = "technical_fi_1"

test_case = "business_fi_1"

results = await test_keyword_analyzer(finnish_texts[test_case], language="fi")


2024-11-23 10:22:57,848 - src.nb_helpers.analyzers - DEBUG - Starting keyword analysis
2024-11-23 10:22:57,851 - src.utils.FileUtils.file_utils - DEBUG [file_utils.py:192] - Initialized FileUtils with log level: INFO
2024-11-23 10:22:57,851 - src.utils.FileUtils.file_utils - DEBUG - Initialized FileUtils with log level: INFO
2024-11-23 10:22:57,853 - src.utils.FileUtils.file_utils - DEBUG [file_utils.py:198] - Project root: c:\Users\tja\OneDrive - Rastor-instituutti ry\Tiedostot\Rastor-instituutti\kehittäminen\analytiikka\repos\semantic-text-analyzer
2024-11-23 10:22:57,853 - src.utils.FileUtils.file_utils - DEBUG - Project root: c:\Users\tja\OneDrive - Rastor-instituutti ry\Tiedostot\Rastor-instituutti\kehittäminen\analytiikka\repos\semantic-text-analyzer
2024-11-23 10:22:57,861 - src.loaders.parameter_handler - DEBUG - No parameter file specified, using defaults
2024-11-23 10:22:57,862 - src.nb_helpers.analyzers - DEBUG - Initialized TextAnalyzer with options: AnalysisOptions(show_co


Keywords Found:
  • liikevaihto          [███████████████████░] (0.95)
  • kasvu                [███████████████████░] (0.95)
  • vuosineljännes       [███████████████████░] (0.95)
  • asiakashankinta      [███████████████████░] (0.95)
  • kustannus            [███████████████████░] (0.95)
  • asiakaspysyvyys      [███████████████████░] (0.95)
  • markkinaosuus        [███████████████████░] (0.95)
  • pilvipalvelu         [███████████████████░] (0.95)

Debug Information:
--------------------
{
  "language": "fi",
  "error": null,
  "success": true,
  "keywords": [
    {
      "keyword": "liikevaihto",
      "score": 0.95,
      "domain": "business",
      "compound_parts": [
        "liike",
        "vaihto"
      ]
    },
    {
      "keyword": "kasvu",
      "score": 0.95,
      "domain": "business",
      "compound_parts": null
    },
    {
      "keyword": "vuosinelj\u00e4nnes",
      "score": 0.95,
      "domain": "business",
      "compound_parts": [
        "vuosi",
        "ne

In [25]:
# from src.nb_helpers.testers import analyze_problematic_words, KeywordTester
# Method 1: Use directly with a processor
# processor = create_text_processor(language="fi")
# problematic_words = ["para", "parani", "parantua", "kasvu", "kasvaa"]
# analyze_problematic_words(processor, problematic_words)

# # Method 2: Use through KeywordTester
# tester = KeywordTester(language_processor=create_text_processor(language="fi"))
# tester.analyze_words(["para", "parani", "parantua", "kasvu", "kasvaa"])

In [26]:
# from src.core.language_processing.finnish import FinnishTextProcessor,analyze_problematic_words
# # Usage example:
# problematic_words = ["para", "parani", "parantua", "kasvu", "kasvaa"]
# analyze_problematic_words(processor, problematic_words)

In [42]:
# test_case = "technical_fi_1"
test_case = "business_fi_1"

results = await test_keyword_analyzer(finnish_texts[test_case], language="fi")

2024-11-23 10:23:04,379 - src.nb_helpers.analyzers - DEBUG - Starting keyword analysis
2024-11-23 10:23:04,383 - src.utils.FileUtils.file_utils - DEBUG [file_utils.py:192] - Initialized FileUtils with log level: INFO
2024-11-23 10:23:04,383 - src.utils.FileUtils.file_utils - DEBUG - Initialized FileUtils with log level: INFO
2024-11-23 10:23:04,386 - src.utils.FileUtils.file_utils - DEBUG [file_utils.py:198] - Project root: c:\Users\tja\OneDrive - Rastor-instituutti ry\Tiedostot\Rastor-instituutti\kehittäminen\analytiikka\repos\semantic-text-analyzer
2024-11-23 10:23:04,386 - src.utils.FileUtils.file_utils - DEBUG - Project root: c:\Users\tja\OneDrive - Rastor-instituutti ry\Tiedostot\Rastor-instituutti\kehittäminen\analytiikka\repos\semantic-text-analyzer
2024-11-23 10:23:04,392 - src.loaders.parameter_handler - DEBUG - No parameter file specified, using defaults
2024-11-23 10:23:04,393 - src.nb_helpers.analyzers - DEBUG - Initialized TextAnalyzer with options: AnalysisOptions(show_co


Keywords Found:
  • liikevaihto          [███████████████████░] (0.95)
  • kasvu                [███████████████████░] (0.95)
  • vuosineljännes       [███████████████████░] (0.95)
  • asiakashankinta      [███████████████████░] (0.95)
  • kustannus            [███████████████████░] (0.95)

Debug Information:
--------------------
{
  "language": "fi",
  "error": null,
  "success": true,
  "keywords": [
    {
      "keyword": "liikevaihto",
      "score": 0.95,
      "domain": "business",
      "compound_parts": [
        "liike",
        "vaihto"
      ]
    },
    {
      "keyword": "kasvu",
      "score": 0.95,
      "domain": "business",
      "compound_parts": null
    },
    {
      "keyword": "vuosinelj\u00e4nnes",
      "score": 0.95,
      "domain": "business",
      "compound_parts": [
        "vuosi",
        "nelj\u00e4nnes"
      ]
    },
    {
      "keyword": "asiakashankinta",
      "score": 0.95,
      "domain": "business",
      "compound_parts": [
        "asiakas",


In [43]:
test_case = "Mixed Domain Content"
results = await test_keyword_analyzer(test_texts[test_case],language="en")


2024-11-23 10:23:24,864 - src.nb_helpers.analyzers - DEBUG - Starting keyword analysis
2024-11-23 10:23:24,868 - src.utils.FileUtils.file_utils - DEBUG [file_utils.py:192] - Initialized FileUtils with log level: INFO
2024-11-23 10:23:24,868 - src.utils.FileUtils.file_utils - DEBUG - Initialized FileUtils with log level: INFO
2024-11-23 10:23:24,871 - src.utils.FileUtils.file_utils - DEBUG [file_utils.py:198] - Project root: c:\Users\tja\OneDrive - Rastor-instituutti ry\Tiedostot\Rastor-instituutti\kehittäminen\analytiikka\repos\semantic-text-analyzer
2024-11-23 10:23:24,871 - src.utils.FileUtils.file_utils - DEBUG - Project root: c:\Users\tja\OneDrive - Rastor-instituutti ry\Tiedostot\Rastor-instituutti\kehittäminen\analytiikka\repos\semantic-text-analyzer
2024-11-23 10:23:24,878 - src.loaders.parameter_handler - DEBUG - No parameter file specified, using defaults
2024-11-23 10:23:24,879 - src.nb_helpers.analyzers - DEBUG - Initialized TextAnalyzer with options: AnalysisOptions(show_co


Keywords Found:
  • cloud-native solutions [███████████████████░] (0.95)
  • monthly recurring revenue [██████████████████░░] (0.90)
  • saas products        [██████████████████░░] (0.90)
  • deployment frequency [█████████████████░░░] (0.85)
  • infrastructure costs [█████████████████░░░] (0.85)

Debug Information:
--------------------
{
  "language": "en",
  "error": null,
  "success": true,
  "keywords": [
    {
      "keyword": "cloud-native solutions",
      "score": 0.95,
      "domain": "technical",
      "compound_parts": [
        "cloud",
        "native solutions"
      ]
    },
    {
      "keyword": "monthly recurring revenue",
      "score": 0.9,
      "domain": "business",
      "compound_parts": [
        "monthly recurring revenue"
      ]
    },
    {
      "keyword": "saas products",
      "score": 0.9,
      "domain": "technical",
      "compound_parts": [
        "saas products"
      ]
    },
    {
      "keyword": "deployment frequency",
      "score": 0.85,
   

In [29]:
# Run tests
for case_name, text in test_texts.items():
    print(f"\nTesting: {case_name}")
    print("=" * 50)
    results = await test_keyword_analyzer(text)

2024-11-23 10:06:59,154 - src.nb_helpers.analyzers - DEBUG - Starting keyword analysis
2024-11-23 10:06:59,158 - src.utils.FileUtils.file_utils - DEBUG [file_utils.py:192] - Initialized FileUtils with log level: INFO
2024-11-23 10:06:59,158 - src.utils.FileUtils.file_utils - DEBUG - Initialized FileUtils with log level: INFO
2024-11-23 10:06:59,161 - src.utils.FileUtils.file_utils - DEBUG [file_utils.py:198] - Project root: c:\Users\tja\OneDrive - Rastor-instituutti ry\Tiedostot\Rastor-instituutti\kehittäminen\analytiikka\repos\semantic-text-analyzer
2024-11-23 10:06:59,161 - src.utils.FileUtils.file_utils - DEBUG - Project root: c:\Users\tja\OneDrive - Rastor-instituutti ry\Tiedostot\Rastor-instituutti\kehittäminen\analytiikka\repos\semantic-text-analyzer
2024-11-23 10:06:59,165 - src.loaders.parameter_handler - DEBUG - No parameter file specified, using defaults
2024-11-23 10:06:59,169 - src.nb_helpers.analyzers - DEBUG - Initialized TextAnalyzer with options: AnalysisOptions(show_co


Testing: Technical Compound Terms


2024-11-23 10:07:00,536 - src.utils.FileUtils.file_utils - DEBUG [file_utils.py:192] - Initialized FileUtils with log level: INFO
2024-11-23 10:07:00,536 - src.utils.FileUtils.file_utils - DEBUG - Initialized FileUtils with log level: INFO
2024-11-23 10:07:00,540 - src.utils.FileUtils.file_utils - DEBUG [file_utils.py:198] - Project root: c:\Users\tja\OneDrive - Rastor-instituutti ry\Tiedostot\Rastor-instituutti\kehittäminen\analytiikka\repos\semantic-text-analyzer
2024-11-23 10:07:00,540 - src.utils.FileUtils.file_utils - DEBUG - Project root: c:\Users\tja\OneDrive - Rastor-instituutti ry\Tiedostot\Rastor-instituutti\kehittäminen\analytiikka\repos\semantic-text-analyzer
2024-11-23 10:07:00,548 - src.nb_helpers.analyzers - DEBUG - Running keyword analysis
2024-11-23 10:07:00,548 - src.nb_helpers.testers - DEBUG - KeywordTester starting analysis
2024-11-23 10:07:00,552 - src.analyzers.keyword_analyzer - DEBUG - Starting keyword analysis for language: en
2024-11-23 10:07:00,552 - src.ana


Keywords Found:
  • cloud migration      [███████████████████░] (0.95)
  • system scalability   [██████████████████░░] (0.90)
  • devops practices     [██████████████████░░] (0.92)
  • deployment pipeline  [█████████████████░░░] (0.88)
  • microservices architecture [██████████████████░░] (0.93)
  • api integrations     [██████████████████░░] (0.91)

Debug Information:
--------------------
{
  "language": "en",
  "error": null,
  "success": true,
  "keywords": [
    {
      "keyword": "cloud migration",
      "score": 0.95,
      "domain": "technical",
      "compound_parts": [
        "cloud migration"
      ]
    },
    {
      "keyword": "system scalability",
      "score": 0.9,
      "domain": "technical",
      "compound_parts": [
        "system scalability"
      ]
    },
    {
      "keyword": "devops practices",
      "score": 0.92,
      "domain": "technical",
      "compound_parts": [
        "devops practices"
      ]
    },
    {
      "keyword": "deployment pipeline",
  

2024-11-23 10:07:05,855 - src.utils.FileUtils.file_utils - DEBUG [file_utils.py:192] - Initialized FileUtils with log level: INFO
2024-11-23 10:07:05,855 - src.utils.FileUtils.file_utils - DEBUG - Initialized FileUtils with log level: INFO
2024-11-23 10:07:05,855 - src.utils.FileUtils.file_utils - DEBUG [file_utils.py:198] - Project root: c:\Users\tja\OneDrive - Rastor-instituutti ry\Tiedostot\Rastor-instituutti\kehittäminen\analytiikka\repos\semantic-text-analyzer
2024-11-23 10:07:05,855 - src.utils.FileUtils.file_utils - DEBUG - Project root: c:\Users\tja\OneDrive - Rastor-instituutti ry\Tiedostot\Rastor-instituutti\kehittäminen\analytiikka\repos\semantic-text-analyzer
2024-11-23 10:07:05,863 - src.nb_helpers.analyzers - DEBUG - Running keyword analysis
2024-11-23 10:07:05,863 - src.nb_helpers.testers - DEBUG - KeywordTester starting analysis
2024-11-23 10:07:05,867 - src.analyzers.keyword_analyzer - DEBUG - Starting keyword analysis for language: en
2024-11-23 10:07:05,867 - src.ana


Keywords Found:
  • cloud-native solutions [███████████████████░] (0.95)
  • monthly recurring revenue [██████████████████░░] (0.90)
  • saas products        [██████████████████░░] (0.92)
  • deployment frequency [█████████████████░░░] (0.88)
  • infrastructure costs [█████████████████░░░] (0.87)

Debug Information:
--------------------
{
  "language": "en",
  "error": null,
  "success": true,
  "keywords": [
    {
      "keyword": "cloud-native solutions",
      "score": 0.95,
      "domain": "technical",
      "compound_parts": [
        "cloud",
        "native solutions"
      ]
    },
    {
      "keyword": "monthly recurring revenue",
      "score": 0.9,
      "domain": "business",
      "compound_parts": [
        "monthly recurring revenue"
      ]
    },
    {
      "keyword": "saas products",
      "score": 0.92,
      "domain": "technical",
      "compound_parts": [
        "saas products"
      ]
    },
    {
      "keyword": "deployment frequency",
      "score": 0.88,
  

2024-11-23 10:07:12,991 - src.utils.FileUtils.file_utils - DEBUG [file_utils.py:192] - Initialized FileUtils with log level: INFO
2024-11-23 10:07:12,991 - src.utils.FileUtils.file_utils - DEBUG - Initialized FileUtils with log level: INFO
2024-11-23 10:07:12,996 - src.utils.FileUtils.file_utils - DEBUG [file_utils.py:198] - Project root: c:\Users\tja\OneDrive - Rastor-instituutti ry\Tiedostot\Rastor-instituutti\kehittäminen\analytiikka\repos\semantic-text-analyzer
2024-11-23 10:07:12,996 - src.utils.FileUtils.file_utils - DEBUG - Project root: c:\Users\tja\OneDrive - Rastor-instituutti ry\Tiedostot\Rastor-instituutti\kehittäminen\analytiikka\repos\semantic-text-analyzer
2024-11-23 10:07:13,001 - src.nb_helpers.analyzers - DEBUG - Running keyword analysis
2024-11-23 10:07:13,001 - src.nb_helpers.testers - DEBUG - KeywordTester starting analysis
2024-11-23 10:07:13,006 - src.analyzers.keyword_analyzer - DEBUG - Starting keyword analysis for language: en
2024-11-23 10:07:13,006 - src.ana


Keywords Found:
  • market analysis      [███████████████████░] (0.95)
  • revenue growth       [███████████████████░] (0.95)
  • customer acquisition costs [███████████████████░] (0.95)
  • retention rates      [██████████████████░░] (0.90)
  • strategic partnerships [██████████████████░░] (0.90)
  • innovation           [█████████████████░░░] (0.85)

Debug Information:
--------------------
{
  "language": "en",
  "error": null,
  "success": true,
  "keywords": [
    {
      "keyword": "market analysis",
      "score": 0.95,
      "domain": "business",
      "compound_parts": [
        "market analysis"
      ]
    },
    {
      "keyword": "revenue growth",
      "score": 0.95,
      "domain": "business",
      "compound_parts": [
        "revenue growth"
      ]
    },
    {
      "keyword": "customer acquisition costs",
      "score": 0.95,
      "domain": "business",
      "compound_parts": [
        "customer acquisition costs"
      ]
    },
    {
      "keyword": "retention ra

2024-11-23 10:07:18,045 - src.utils.FileUtils.file_utils - DEBUG [file_utils.py:192] - Initialized FileUtils with log level: INFO
2024-11-23 10:07:18,045 - src.utils.FileUtils.file_utils - DEBUG - Initialized FileUtils with log level: INFO
2024-11-23 10:07:18,045 - src.utils.FileUtils.file_utils - DEBUG [file_utils.py:198] - Project root: c:\Users\tja\OneDrive - Rastor-instituutti ry\Tiedostot\Rastor-instituutti\kehittäminen\analytiikka\repos\semantic-text-analyzer
2024-11-23 10:07:18,045 - src.utils.FileUtils.file_utils - DEBUG - Project root: c:\Users\tja\OneDrive - Rastor-instituutti ry\Tiedostot\Rastor-instituutti\kehittäminen\analytiikka\repos\semantic-text-analyzer
2024-11-23 10:07:18,054 - src.nb_helpers.analyzers - DEBUG - Running keyword analysis
2024-11-23 10:07:18,054 - src.nb_helpers.testers - DEBUG - KeywordTester starting analysis
2024-11-23 10:07:18,054 - src.analyzers.keyword_analyzer - DEBUG - Starting keyword analysis for language: en
2024-11-23 10:07:18,054 - src.ana


Keywords Found:
  • machine learning     [███████████████████░] (0.95)
  • real-time data streams [██████████████████░░] (0.90)
  • ci/cd pipeline       [██████████████████░░] (0.92)
  • automated testing workflows [█████████████████░░░] (0.88)
  • cloud-based infrastructure [██████████████████░░] (0.91)
  • multi-region deployments [█████████████████░░░] (0.89)

Debug Information:
--------------------
{
  "language": "en",
  "error": null,
  "success": true,
  "keywords": [
    {
      "keyword": "machine learning",
      "score": 0.95,
      "domain": "technical",
      "compound_parts": [
        "machine learning"
      ]
    },
    {
      "keyword": "real-time data streams",
      "score": 0.9,
      "domain": "technical",
      "compound_parts": [
        "real",
        "time data streams"
      ]
    },
    {
      "keyword": "ci/cd pipeline",
      "score": 0.92,
      "domain": "technical",
      "compound_parts": [
        "ci/cd pipeline"
      ]
    },
    {
      "keywo

previous examples

In [ ]:
# Test logging
logger = logging.getLogger("src.analyzers.keyword_analyzer")
logger.debug("Testing keyword analyzer logging")

NOTE: following are not working with the new parameter handling model

In [44]:
from src.nb_helpers.excel import analyze_excel_content

In [45]:
# Example texts in different languages
example_texts = {
    "English Technical": """
        The cloud migration project improved system scalability while reducing costs.
        New DevOps practices streamlined the deployment pipeline significantly.
    """,
    
    "Finnish Technical": """
        Pilvipalveluihin siirtyminen paransi järjestelmän skaalautuvuutta ja vähensi kustannuksia.
        Uudet DevOps-käytännöt tehostivat merkittävästi käyttöönottoprosessia.
    """
}

# # Analyze with automatic language detection
# for name, text in example_texts.items():
#     print(f"\nAnalyzing {name}:")
#     results = await analyze_text(text, options)

# Example with specific language and parameters
fi_options = AnalysisOptions(
    show_confidence=True,
    show_evidence=True,
    debug_mode=True,
    language="fi",
    parameter_file="parameters_fi.xslx"
)

# Analyze Finnish text with specific parameters
fi_results = await analyze_text(example_texts["Finnish Technical"], fi_options)

# # Batch process Excel file with language detection
# await analyze_excel_content(
#     input_file="multilingual_texts.xlsx",
#     output_file="analysis_results",
#     content_column="content",
#     parameter_file="analysis_params.yaml",
#     language_column="language"  # Optional column specifying language
# )

2024-11-23 10:31:11,990 - src.nb_helpers.analyzers - DEBUG - Starting complete text analysis
2024-11-23 10:31:11,993 - src.nb_helpers.analyzers - DEBUG - Running keywords analysis
2024-11-23 10:31:11,994 - src.nb_helpers.analyzers - DEBUG - Starting keyword analysis
2024-11-23 10:31:11,998 - src.utils.FileUtils.file_utils - DEBUG [file_utils.py:192] - Initialized FileUtils with log level: INFO
2024-11-23 10:31:11,998 - src.utils.FileUtils.file_utils - DEBUG - Initialized FileUtils with log level: INFO
2024-11-23 10:31:12,000 - src.utils.FileUtils.file_utils - DEBUG [file_utils.py:198] - Project root: c:\Users\tja\OneDrive - Rastor-instituutti ry\Tiedostot\Rastor-instituutti\kehittäminen\analytiikka\repos\semantic-text-analyzer
2024-11-23 10:31:12,000 - src.utils.FileUtils.file_utils - DEBUG - Project root: c:\Users\tja\OneDrive - Rastor-instituutti ry\Tiedostot\Rastor-instituutti\kehittäminen\analytiikka\repos\semantic-text-analyzer
2024-11-23 10:31:12,006 - src.loaders.parameter_handl


Keywords Found:
  • pilvipalvelu         [███████████████████░] (0.95)
  • skaalautuvuus        [███████████████████░] (0.95)
  • kustannus            [███████████████████░] (0.95)
  • merkittävä           [██████████████████░░] (0.90)
  • käyttöönottoprosessi [███████████████████░] (0.95)

Debug Information:
--------------------
{
  "language": "fi",
  "error": null,
  "success": true,
  "keywords": [
    {
      "keyword": "pilvipalvelu",
      "score": 0.95,
      "domain": "technical",
      "compound_parts": [
        "pilvi",
        "palvella"
      ]
    },
    {
      "keyword": "skaalautuvuus",
      "score": 0.95,
      "domain": "technical",
      "compound_parts": null
    },
    {
      "keyword": "kustannus",
      "score": 0.95,
      "domain": "business",
      "compound_parts": null
    },
    {
      "keyword": "merkitt\u00e4v\u00e4",
      "score": 0.9,
      "domain": "technical",
      "compound_parts": null
    },
    {
      "keyword": "k\u00e4ytt\u00f6\u00f6no

2024-11-23 10:31:17,593 - src.utils.FileUtils.file_utils - DEBUG [file_utils.py:192] - Initialized FileUtils with log level: INFO
2024-11-23 10:31:17,593 - src.utils.FileUtils.file_utils - DEBUG - Initialized FileUtils with log level: INFO
2024-11-23 10:31:17,596 - src.utils.FileUtils.file_utils - DEBUG [file_utils.py:198] - Project root: c:\Users\tja\OneDrive - Rastor-instituutti ry\Tiedostot\Rastor-instituutti\kehittäminen\analytiikka\repos\semantic-text-analyzer
2024-11-23 10:31:17,596 - src.utils.FileUtils.file_utils - DEBUG - Project root: c:\Users\tja\OneDrive - Rastor-instituutti ry\Tiedostot\Rastor-instituutti\kehittäminen\analytiikka\repos\semantic-text-analyzer
2024-11-23 10:31:17,602 - src.nb_helpers.analyzers - DEBUG - Running theme analysis
2024-11-23 10:31:22,921 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-11-23 10:31:22,928 - src.nb_helpers.analyzers - DEBUG - Theme analysis completed
2024-11-23 10:31:22,929 - src


Themes Found:

  • Cloud Services Transition
    Confidence: [███████████████████░] (0.95)
    The shift to cloud services improved system scalability and reduced costs, indicating a strategic move towards more efficient resource management.

  • DevOps Practices Enhancement
    Confidence: [██████████████████░░] (0.90)
    The implementation of new DevOps practices significantly streamlined the deployment process, showcasing the importance of modern methodologies in software development.

Debug Information:
--------------------
{
  "language": "unknown",
  "error": null,
  "success": true,
  "themes": [
    {
      "name": "Cloud Services Transition",
      "description": "The shift to cloud services improved system scalability and reduced costs, indicating a strategic move towards more efficient resource management.",
      "confidence": 0.95,
      "keywords": [
        "cloud services",
        "scalability",
        "cost reduction"
      ],
      "parent_theme": null
    },
    

2024-11-23 10:31:24,307 - src.utils.FileUtils.file_utils - DEBUG [file_utils.py:192] - Initialized FileUtils with log level: INFO
2024-11-23 10:31:24,307 - src.utils.FileUtils.file_utils - DEBUG - Initialized FileUtils with log level: INFO
2024-11-23 10:31:24,310 - src.utils.FileUtils.file_utils - DEBUG [file_utils.py:198] - Project root: c:\Users\tja\OneDrive - Rastor-instituutti ry\Tiedostot\Rastor-instituutti\kehittäminen\analytiikka\repos\semantic-text-analyzer
2024-11-23 10:31:24,310 - src.utils.FileUtils.file_utils - DEBUG - Project root: c:\Users\tja\OneDrive - Rastor-instituutti ry\Tiedostot\Rastor-instituutti\kehittäminen\analytiikka\repos\semantic-text-analyzer
2024-11-23 10:31:24,317 - src.core.language_processing.factory - DEBUG - Using default configuration
2024-11-23 10:31:24,318 - src.core.language_processing.factory - DEBUG - Creating text processor for language: None
2024-11-23 10:31:24,319 - src.core.language_processing.factory - DEBUG - Creating text processor for la


Categories Found:

  • technical
    Confidence: [█████████████████░░░] (0.85)


AttributeError: 'CategoryMatch' object has no attribute 'explanation'

<!-- ## Analysis Functions

### Single Analysis with Debug Output
Run detailed analysis for a single text: -->


In [ ]:
example_texts = {
    "Business Analysis": """
        Q3 revenue increased by 15% with strong growth in enterprise sales.
        Customer retention improved while acquisition costs decreased.
        New market expansion initiatives are showing positive early results.
    """,
    
    "Technical Content": """
        The application uses microservices architecture with containerized deployments.
        Data processing pipeline incorporates machine learning models for prediction.
        System monitoring ensures high availability and performance metrics.
    """,
    
    "Mixed Content": """
        The IT department's cloud migration project reduced infrastructure costs by 25%.
        DevOps implementation improved deployment frequency while maintaining quality.
        Monthly recurring revenue from SaaS products grew steadily.
    """,
    "koulutus":
    """
        Verkko-oppimisalusta sisältää interaktiivisia moduuleja ja oman tahdin edistymisen seurannan. 
        Virtuaaliluokat mahdollistavat reaaliaikaisen yhteistyön opiskelijoiden ja ohjaajien välillä. 
        Digitaaliset arviointityökalut antavat välitöntä palautetta oppimistuloksista.
    """,
    "tekninen":
    """
        Koneoppimismalleja koulutetaan suurilla datajoukolla tunnistamaan kaavoja. 
        Neuroverkon arkkitehtuuri sisältää useita kerroksia piirteiden erottamiseen. 
        Datan esikäsittely ja piirteiden suunnittelu ovat keskeisiä vaiheita prosessissa.

    """
}